In [1]:
# libraries for nlp
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# libraries required for Tensorflow processing
import tensorflow as tf
import tflearn
import numpy as np
import json
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADITYA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
W0817 13:57:42.146343  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\helpers\summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0817 13:57:42.161944  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\helpers\trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0817 13:57:42.193113  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\local\programs\python\python37\lib\site-packages\tflearn\collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0817 13:57:42.224346  8036 deprecation_wrapper.py:119] From c:\users\aditya\appdata\

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
print(tf.__version__)

1.14.0


In [8]:
with open('intents.json') as json_data:
    intents = json.load(json_data)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Hello', 'Is anyone there?', 'Good day'],
   'responses': ['Hi there, how can I help?',
    'Hello, thanks for visiting',
    'Good to see you again'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Goodbye', 'Bye', 'See you later'],
   'responses': ['Have a nice day',
    'See you later, thanks for visiting',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thank you', 'Thanks', "That's helpful"],
   'responses': ['No problem', 'My pleasure', 'Happy to help!', 'Any time!']},
  {'tag': 'hours',
   'patterns': ['When are you open?',
    'What hours are you open?',
    'What are your hours?'],
   'responses': ['Our hours are 9am-5pm every day',
    "We're open every day 9am-5pm"]},
  {'tag': 'location',
   'patterns': ['Where are you located?',
    'What is your address?',
    'What is the location of your clinic?'],
   'responses': ['Our address is Connaught Place New Delhi',
    

In [64]:
type(intents)

dict

In [65]:
type(intents['intents'])

list

In [67]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence present in 'patterns' each element of 'intents' list 
for intent in intents['intents']:
    # loop through each element of every 'patterns' list present inside the 'intents' list
    for pattern in intent['patterns']:
        # create tokens for each word in every element of the list 
        w = nltk.word_tokenize(pattern)
        # add to the words list
        words.extend(w) # extend list by appending elements from the iterable i.e. list(w)
        documents.append((w, intent['tag'])) # add to the list the tuple containing list of tokens and the corresponding tag
        # add the tag to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [68]:
words

['Hi',
 'How',
 'are',
 'you',
 'Hello',
 'Is',
 'anyone',
 'there',
 '?',
 'Good',
 'day',
 'Goodbye',
 'Bye',
 'See',
 'you',
 'later',
 'Thank',
 'you',
 'Thanks',
 'That',
 "'s",
 'helpful',
 'When',
 'are',
 'you',
 'open',
 '?',
 'What',
 'hours',
 'are',
 'you',
 'open',
 '?',
 'What',
 'are',
 'your',
 'hours',
 '?',
 'Where',
 'are',
 'you',
 'located',
 '?',
 'What',
 'is',
 'your',
 'address',
 '?',
 'What',
 'is',
 'the',
 'location',
 'of',
 'your',
 'clinic',
 '?',
 'Are',
 'you',
 'cash',
 'only',
 '?',
 'Do',
 'you',
 'take',
 'credit',
 'cards',
 '?',
 'Do',
 'you',
 'accept',
 'Mastercard',
 '?',
 'Do',
 'you',
 'take',
 'payments',
 'through',
 'mobile',
 'wallets',
 'Are',
 'you',
 'open',
 'today',
 '?',
 'When',
 'do',
 'you',
 'open',
 'today',
 '?',
 'What',
 'are',
 'your',
 'hours',
 'today',
 '?',
 'What',
 'are',
 'the',
 'tests',
 'done',
 '?',
 'Which',
 'tests',
 'are',
 'available',
 'at',
 'the',
 'clinic',
 '?',
 'What',
 'are',
 'services',
 'availabl

In [69]:
classes

['greeting',
 'goodbye',
 'thanks',
 'hours',
 'location',
 'payments',
 'opentoday',
 'tests',
 'today']

In [70]:
documents

[(['Hi'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Goodbye'], 'goodbye'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Thank', 'you'], 'thanks'),
 (['Thanks'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['When', 'are', 'you', 'open', '?'], 'hours'),
 (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'),
 (['What', 'are', 'your', 'hours', '?'], 'hours'),
 (['Where', 'are', 'you', 'located', '?'], 'location'),
 (['What', 'is', 'your', 'address', '?'], 'location'),
 (['What', 'is', 'the', 'location', 'of', 'your', 'clinic', '?'], 'location'),
 (['Are', 'you', 'cash', 'only', '?'], 'payments'),
 (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'),
 (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'),
 (['Do', 'you', 'take', 'payments', 'through', 'mobile', 'wallets'],
  'payments'),
 (['Are', 'you', 'open', 'today', '?'], 'ope

In [126]:
# perform stemming on every word and remove duplicate words using set
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicate classes using set
classes = sorted(list(set(classes)))

print ("\nLength of 'documents' list: ", len(documents))
print ("\nLength of 'classes' list: ", len(classes))
print ("\nLength of unique stemmed words list: ", len(words), "\nStemmed unique words:\n", words)


Length of 'documents' list:  28

Length of 'classes' list:  9

Length of unique stemmed words list:  48 
Stemmed unique words:
 ["'s", 'acceiv', 'address', 'anyon', 'ar', 'at', 'avail', 'bye', 'card', 'cash', 'clin', 'credit', 'day', 'do', 'don', 'good', 'goodby', 'hello', 'help', 'hi', 'hour', 'how', 'is', 'lat', 'loc', 'mastercard', 'mobl', 'of', 'on', 'op', 'pay', 'see', 'serv', 'tak', 'test', 'thank', 'that', 'the', 'ther', 'through', 'today', 'wallet', 'what', 'when', 'wher', 'which', 'yo', 'you']


In [109]:
# create a list to store the training data
training = []
output = []
# create an empty array for output of length equal to the number of unique classes/tags 
output_empty = [0] * len(classes)

# create bag of words for each sentence in each 'patterns' list which has been stored in 'documents' list 
# as a tuple of list of words in the sentence and class of the sentence.  
# Create training list by appending list of 'bag of words' list and output list.  
# 'classes' is a list of 'tags'. In 'output_row' list, '1' is present only at the index corresponding to index in 'classes'
# of a tag to which bag of words belongs. The rest elements are '0' in 'output_row'   
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create the bag of words list and append it to training list along with the current tag as a  
    # For a list 'bag' of the size equal to 'words' list, the words present in 'doc' i.e. each sentence of 'patterns'
    # is 1 in bag rest corresponding indexes are 0 if a word in 'words' is not present in the 'pattern_words' list
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output '1' for current tag and '0' for other tags 
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle the features and turn it into a numpy array
random.shuffle(training)
training = np.array(training)

# create train and test data
train_x = list(training[:,0])
train_y = list(training[:,1])

In [110]:
# reset underlying graph data
tf.reset_default_graph()

In [111]:
# build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

In [112]:
# define model, setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# train the model with gradient descent algorithm
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 1.07596 | time: 0.016s
| Adam | epoch: 1000 | loss: 1.07596 - acc: 0.9078 -- iter: 24/28
Training Step: 4000  | total loss: 0.98700 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.98700 - acc: 0.9170 -- iter: 28/28
--


In [113]:
# save all the data structures
import pickle
pickle.dump( {'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open( "training_data", "wb" ) )

In [114]:
# restore all the data structures
data = pickle.load( open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [115]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [116]:
# load the saved model
model.load('./model.tflearn')

In [117]:
def clean_up_sentence(sentence):
    # tokenize the sentences
    token_words = nltk.word_tokenize(sentence)
    # stem each word present as tokens
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    return token_words

In [118]:
# generate 'bag of words' array containg 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the sentences
    token_words = clean_up_sentence(sentence)
    # generate bag of words
    bag = [0]*len(words)  
    for s in token_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("Found in bag: %s" % w)

    return(np.array(bag))

In [151]:
s = bow("what is the location of your clinic?", words)
print(s)

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 1 0]


In [154]:
prob_threshold = 0.2 # filter out any prediction having the probability less than 0.3
def classify(sentence):
    # generate probabilities using model.predict
    probs = model.predict([bow(sentence, words)])[0]
    # store the predictions which are above the threshold and filter out the rest
    probs = [[i,p] for i, p in enumerate(probs) if p > prob_threshold]
    # sort the probabilities in descending order
    probs.sort(key = lambda x: x[1], reverse = True)
    result = []
    for p in probs:
        result.append((classes[p[0]], p[1])) #  append the list with a tuple containing tag's index and its probability
    # return the tuple
    return result
